In [1]:
import coffea
from git import Repo
import awkward as ak
import uproot
import dask_awkward as dak
mod = "Prayag Yadav"
local_repo = Repo(path='coffea')
local_branch = local_repo.active_branch.name
print("_______________________________________")
print("\tCurrent Configuration")
print("---------------------------------------")
print("Coffea Version: ", coffea.__version__)
print("Branch: \t", local_branch)
print("Modified by: \t", mod)
print("_______________________________________")


_______________________________________
	Current Configuration
---------------------------------------
Coffea Version:  0.1.dev3583+ge06c4b8
Branch: 	 master
Modified by: 	 Prayag Yadav
_______________________________________


In [2]:
from coffea.nanoevents import NanoEventsFactory, FCCSchema, FCC
test_file = '../../../coffea-fcc-analyses/data/p8_ee_ZH_ecm240/events_082532938.root'
file = uproot.open(test_file)

events = NanoEventsFactory.from_root(
    test_file+":events",
    entry_stop=100,
    schemaclass= FCC.get_schema(version="latest"),
    delayed = False,
    metadata=file["metadata"].arrays()
).events()

file.close()

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/schemas/fcc.py:4: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [3]:
events.metadata

<Array [{m_collectionIDs: [...], ...}] type='1 * {m_collectionIDs: var * in...'>

In [4]:
events.metadata.m_names

<Array [['Electron', 'Muon', ..., 'Jet', 'EFlowTrack']] type='1 * var * string'>

In [5]:
events.metadata.m_collectionIDs

<Array [[1, 2, 3, 4, 5, 6, ..., 8, 9, 10, 11, 12, 13]] type='1 * var * int32'>

In [6]:
print("Collection ID \t Collection Name")
print("---------------------------------------")
for name, id in zip(events.metadata.m_names[0], events.metadata.m_collectionIDs[0]):
    print(id, '\t\t', name)

Collection ID 	 Collection Name
---------------------------------------
1 		 Electron
2 		 Muon
3 		 AllMuon
4 		 EFlowNeutralHadron
5 		 Particle
6 		 Photon
7 		 ReconstructedParticles
8 		 EFlowPhoton
9 		 MCRecoAssociations
10 		 MissingET
11 		 ParticleIDs
12 		 Jet
13 		 EFlowTrack


In [7]:
events.ReconstructedParticles.fields

['E',
 'charge',
 'clusters',
 'covMatrix_10_',
 'goodnessOfPID',
 'mass',
 'particleIDs',
 'particles',
 'px',
 'py',
 'pz',
 'referencePoint',
 'tracks',
 'type']

In [8]:
def tree(Record_Array):
    '''
    To list down all the fields and subfields of a ListOffsetArray with RecordArray members
    '''
    f,l,d = ak.to_buffers(Record_Array)
    f = f.to_dict()
    if f["class"] == "ListOffsetArray":
        c = f["content"]
    elif f["class"] == "RecordArray":
        c = f["contents"]
    else :
        raise TypeError("Incompatible class type of the input array.")
    if c["class"] == "RecordArray":
        fields = c["fields"]
        for num, field in enumerate(fields):
            print("*", field)
            s = c["contents"][num]
            if s["class"] == "RecordArray":
                sub_fields = s["fields"]
                for sub_field in sub_fields:
                    print("\t-",sub_field)
                print()
        print("\n")

In [9]:
print("Fields and Sub-fields are:")
print("--------------------------\n")
tree(events.ReconstructedParticles)

Fields and Sub-fields are:
--------------------------

* E
* charge
* clusters
	- begin
	- end

* covMatrix_10_
* goodnessOfPID
* mass
* particleIDs
	- begin
	- end

* particles
	- begin
	- end

* px
* py
* pz
* referencePoint
	- x
	- y
	- z

* tracks
	- begin
	- end

* type




In [10]:
print("Fields and Sub-fields are:")
print("--------------------------\n")
tree(events.Particle)

Fields and Sub-fields are:
--------------------------

* PDG
* charge
* colorFlow
	- a
	- b

* daughters
	- begin
	- end

* endpoint
	- x
	- y
	- z

* generatorStatus
* mass
* momentumAtEndpoint
	- x
	- y
	- z

* parents
	- begin
	- end

* px
* py
* pz
* simulatorStatus
* spin
	- x
	- y
	- z

* time
* vertex
	- x
	- y
	- z





In [11]:
for f in events.fields:
    print('__________________________')
    print(f)
    print('__________________________\n')
    try:
        tree(events[f])
    except:
        pass

__________________________
AllMuonidx0
__________________________

* collectionID
* index


__________________________
EFlowNeutralHadron
__________________________

* E
* clusters
	- begin
	- end

* directionError
	- x
	- y
	- z

* energyError
* hitContributions
	- begin
	- end

* hits
	- begin
	- end

* iTheta
* particleIDs
	- begin
	- end

* phi
* position
	- x
	- y
	- z

* positionError_6_
* shapeParameters
	- begin
	- end

* subdetectorEnergies
	- begin
	- end

* type


__________________________
EFlowNeutralHadron_0
__________________________

__________________________
EFlowNeutralHadron_1
__________________________

__________________________
EFlowNeutralHadron_2
__________________________

__________________________
EFlowNeutralHadronidx0
__________________________

* collectionID
* index


__________________________
EFlowNeutralHadronidx1
__________________________

* collectionID
* index


__________________________
EFlowNeutralHadronidx2
__________________________

* collec

In [12]:
events.ReconstructedParticles.referencePoint

<ThreeVectorArray [[{x: 0, y: 0, z: 0}, ..., {...}], ...] type='100 * var *...'>

In [13]:
events.ReconstructedParticles.referencePoint.fields

['x', 'y', 'z']

In [14]:
events.ReconstructedParticles.referencePoint.phi 

<Array [[0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0], ...] type='100 * var * float32'>